In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer


In [5]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [6]:
# preprocess & split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [7]:
# define models
models = {
    "Logistic Regression": LogisticRegression(),
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier()
}

In [8]:
# train & evaluate model

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    })


In [9]:
df_results = pd.DataFrame(results)
print("Initial Model Performance:\n", df_results)

Initial Model Performance:
                  Model  Accuracy  Precision    Recall  F1 Score
0  Logistic Regression  0.973684   0.972222  0.985915  0.979021
1                  SVM  0.973684   0.972222  0.985915  0.979021
2        Decision Tree  0.938596   0.957143  0.943662  0.950355
3        Random Forest  0.964912   0.958904  0.985915  0.972222


In [10]:
# Hyperparameter tuning

# Random Forest - GridSearchCV
param_grid_rf = {
    "n_estimators": [50, 100],
    "max_depth": [4, 6, 8],
    "min_samples_split": [2, 5]
}

In [11]:
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='f1')
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [4, 6, 8], 'min_samples_split': [2, 5],
                         'n_estimators': [50, 100]},
             scoring='f1')

In [12]:
# SVM - RandomizedSearchCV
param_dist_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}


In [13]:
rand_svc = RandomizedSearchCV(SVC(), param_distributions=param_dist_svc, n_iter=5, scoring='f1', cv=5, random_state=42)
rand_svc.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=5,
                   param_distributions={'C': [0.1, 1, 10],
                                        'gamma': ['scale', 'auto'],
                                        'kernel': ['linear', 'rbf']},
                   random_state=42, scoring='f1')

In [14]:
# evaluate the tuned models
tuned_models = {
    "Random Forest (Tuned)": grid_rf.best_estimator_,
    "SVM (Tuned)": rand_svc.best_estimator_
}

In [15]:
for name, model in tuned_models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred)
    })

In [16]:
df_final = pd.DataFrame(results)
print("\nFinal Model Performance after Hyperparameter Tuning:\n", df_final.sort_values(by="F1 Score", ascending=False))


Final Model Performance after Hyperparameter Tuning:
                    Model  Accuracy  Precision    Recall  F1 Score
5            SVM (Tuned)  0.982456   0.972603  1.000000  0.986111
0    Logistic Regression  0.973684   0.972222  0.985915  0.979021
1                    SVM  0.973684   0.972222  0.985915  0.979021
3          Random Forest  0.964912   0.958904  0.985915  0.972222
4  Random Forest (Tuned)  0.956140   0.958333  0.971831  0.965035
2          Decision Tree  0.938596   0.957143  0.943662  0.950355


In [17]:
param_grid = {
  'C': [0.1, 1, 10],
  'kernel': ['linear', 'rbf', 'poly'],
  'gamma': [0.1, 1, 'scale', 'auto']
}
grid = GridSearchCV(SVC(), param_grid, cv=5)
grid.fit(X_train, y_train)
best = grid.best_estimator_


In [18]:
from scipy.stats import uniform
param_dist = {
  'C': uniform(0.1, 10),
  'kernel': ['linear','rbf','poly'],
  'gamma': ['scale','auto'] + list(np.logspace(-3,3,50))
}
rand = RandomizedSearchCV(SVC(), param_dist, n_iter=20, cv=5)
rand.fit(X_train, y_train)
best_rand = rand.best_estimator_
